In [81]:
from collections import OrderedDict
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import hijri_converter
from hijri_converter import Hijri, Gregorian

In [82]:
def hijri_to_greg(hijri_date):
    dt_greg = hijri_date.to_gregorian().ctime()
    dt_greg = pd.to_datetime(dt_greg)
    return dt_greg



In [83]:

from hijri_converter import convert
def convert_date(date_string):
    return pd.to_datetime(date_string, errors = 'coerce')

df1 = pd.read_excel("./monthly_target_dates_df_1442.xlsx" , converters={'Reservation Check-out Date':lambda x:Hijri.fromisoformat(x) ,'Reservation Creation Date':lambda x:Hijri.fromisoformat(x),'Reservation Check-in Date':lambda x:Hijri.fromisoformat(x) })
df2 = pd.read_excel("./monthly_target_dates_df_1444.xlsx" ,  converters={'Reservation Check-out Date':lambda x:Hijri.fromisoformat(x) ,'Reservation Creation Date':lambda x:Hijri.fromisoformat(x),'Reservation Check-in Date':lambda x:Hijri.fromisoformat(x)})

In [84]:
df2.drop(df2.index[-1], inplace=True)

In [85]:
df2.insert(1, 'Reservation Creation Date (Greg)', df2['Reservation Creation Date'].apply(lambda x: hijri_to_greg(x)))


In [86]:
def display_full_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

In [87]:
df2.drop(columns=['Reservation Creation Date'], inplace=True)

In [88]:
df2_line = df2.set_index('Reservation Creation Date (Greg)', inplace=False)
#greg_daily_rows_weekly_df.drop(columns=greg_daily_rows_weekly_df.columns[:2], inplace=True)


In [89]:
df2_line

,M1 - Ramadhan,M2 - Shawwal,M3 - Dhu al-Qi’dah,M4 - Dhu al-Hijjah
Reservation Creation Date (Greg),,,,
2022-07-30,0.0,0.0,0.0,0.0
2022-07-31,0.0,0.0,0.0,0.0
2022-08-01,0.0,0.0,0.0,0.0
2022-08-02,0.0,0.0,0.0,0.0
2022-08-03,0.0,0.0,0.0,0.0
...,...,...,...,...
2023-07-14,NaN,NaN,NaN,645.0
2023-07-15,NaN,NaN,NaN,645.0
2023-07-16,NaN,NaN,NaN,NaN


In [90]:
px.line(df2_line)

In [91]:
df2 = df2.melt(id_vars=['Reservation Creation Date (Greg)'], var_name='Target Month', value_name='#Rooms')

In [92]:
df2.fillna(0, inplace=True)
df2

,Reservation Creation Date (Greg),Target Month,#Rooms
0,2022-07-30,M1 - Ramadhan,0.0
1,2022-07-31,M1 - Ramadhan,0.0
2,2022-08-01,M1 - Ramadhan,0.0
3,2022-08-02,M1 - Ramadhan,0.0
4,2022-08-03,M1 - Ramadhan,0.0
...,...,...,...
1411,2023-07-14,M4 - Dhu al-Hijjah,645.0
1412,2023-07-15,M4 - Dhu al-Hijjah,645.0
1413,2023-07-16,M4 - Dhu al-Hijjah,0.0
1414,2023-07-17,M4 - Dhu al-Hijjah,0.0


In [93]:
# display_full_df(df2) 

Bubble chart 

In [94]:
import plotly.express as px

fig = px.scatter(df2, x="Reservation Creation Date (Greg)", y='Target Month',
	         size="#Rooms", color='Target Month',
                 hover_name='Target Month', log_x=False, size_max=80)
fig.show()


Filled Area Plot

In [95]:
import plotly.graph_objects as go


In [97]:
import plotly.express as px

fig = px.area(df2, x="Reservation Creation Date (Greg)", y="#Rooms", color="Target Month",
             pattern_shape="Target Month", pattern_shape_sequence=[".", "x", "+","/"] )

#fig.update_layout(barmode='overlay')

fig.show()

In [98]:
from plotly_calplot import calplot

In [100]:
fig = calplot(
    df2,
    x="Reservation Creation Date (Greg)",
    y="#Rooms",
    gap=0
)
fig.show()